In [ ]:
#Cell 1: Paths & Helper Functions
# Set up paths and import required libraries
# %% 1 · PATHS & HELPERS
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"   # allow duplicate OpenMP runtimes
from pathlib import Path
import pandas as pd, shutil, cv2, numpy as np, random, json
from tqdm import tqdm

ROOT = Path(r"C:/Users/BHARATH SHIVA/Downloads/lpr_project")

TRAIN1_IMG_DIR = ROOT/"train1_images"
TRAIN2_IMG_DIR = ROOT/"train2_images"
TEST_IMG_DIR   = ROOT/"test_images"

# pick the first file that matches either extension
TRAIN1_CSV = next((ROOT/f for f in ["train1_labels.csv","train1_labels.xlsx"] if (ROOT/f).exists()))
TRAIN2_CSV = next((ROOT/f for f in ["train2_labels.csv","train2_labels.xlsx"] if (ROOT/f).exists()))

YOLO_DIR = ROOT/"yolo_dataset"
OCR_DIR  = ROOT/"ocr_dataset"
PRED_CSV = ROOT/"test_predictions.csv"

YOLO_DIR.mkdir(parents=True, exist_ok=True)
(OCR_DIR/"images").mkdir(parents=True, exist_ok=True)

def read_table(path: Path) -> pd.DataFrame:
    return pd.read_excel(path) if path.suffix==".xlsx" else pd.read_csv(path)

print("Paths OK")


# %% 1‑B · QUICK SANITY‑CHECK
for tag, p in [("train1", TRAIN1_IMG_DIR),
               ("train2", TRAIN2_IMG_DIR),
               ("test",   TEST_IMG_DIR),
               ("det_lbl",TRAIN1_CSV),
               ("ocr_lbl",TRAIN2_CSV)]:
    print(f"{tag:<8} →", "OK" if p.exists() else "NOT FOUND")



In [ ]:
#Cell 2: Sanity Check – Verify Input Files
# Quick check to ensure all required paths and files exist
# %% 2 · CONVERT train1_labels → YOLO TXT
df_det = read_table(TRAIN1_CSV)  # must have: filename,ymin,xmin,ymax,xmax
if not {"filename","ymin","xmin","ymax","xmax"}.issubset(df_det.columns):
    raise ValueError("train1_labels missing required columns")

for p in ["images/train","images/val","labels/train","labels/val"]:
    (YOLO_DIR/p).mkdir(parents=True, exist_ok=True)

def to_yolo(size, box):
    h,w = size
    y1,x1,y2,x2 = box
    return [0, (x1+x2)/2/w, (y1+y2)/2/h, (x2-x1)/w, (y2-y1)/h]

val_ratio = 0.1
val_set = set(random.sample(list(df_det.filename.unique()),
                            int(len(df_det)*val_ratio)))

for _,r in tqdm(df_det.iterrows(), total=len(df_det)):
    src = TRAIN1_IMG_DIR/r.filename
    if not src.exists(): continue
    im = cv2.imread(str(src)); h,w = im.shape[:2]
    yolo_line = " ".join(map(str, to_yolo((h,w), r[["ymin","xmin","ymax","xmax"]])))
    split = "val" if r.filename in val_set else "train"
    shutil.copy(src, YOLO_DIR/f"images/{split}"/r.filename)
    with open(YOLO_DIR/f"labels/{split}"/(src.stem+".txt"),"w") as f:
        f.write(yolo_line+"\n")

(YOLO_DIR/"data.yaml").write_text(json.dumps({
    "path": str(YOLO_DIR),
    "train":"images/train",
    "val":"images/val",
    "nc":1,
    "names":{0:"license_plate"}}, indent=2))
print("✔ YOLO data prepared")



In [ ]:
#Cell 3: Convert Detection Labels to YOLO Format
# Load detection labels for YOLO (bounding boxes)
# %% 3 · LIGHT‑WEIGHT YOLO‑v8 DETECTOR TRAIN
from ultralytics import YOLO, checks
from pathlib import Path
checks()  # prints system info

detector = YOLO("yolov8n.pt")  # nano weights

detector.train(
    data    = str(YOLO_DIR / "data.yaml"),
    epochs  = 15,          # reduce epochs for quick run
    imgsz   = 416,         # smaller resolution
    batch   = 4,           # small batch fits RAM
    workers = 0,           # Windows: avoid multi‑proc crashes
    freeze  = 10,          # train only detection head
    plots   = False,       # skip heavy visualizations
    name    = "lp_det_safe"
)

best_det = Path(detector.trainer.save_dir) / "best.pt"
print("✔ Detector trained:", best_det)


In [ ]:
# Cell 4: Train YOLOv8 Detection Model
# Train YOLOv8 on detection dataset
# %% 4 · PREP OCR DATASET & TRAIN CRNN  (fixed height‑pool bug)
import shutil, cv2, numpy as np, torch, torch.nn as nn, torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch.nn.functional as F

# ---------- A · load labels & copy images ----------
df_ocr = read_table(TRAIN2_CSV)
if "img_id" not in df_ocr.columns:
    df_ocr.columns = ["img_id", "text"]

(OCR_DIR/"images").mkdir(parents=True, exist_ok=True)
for f in tqdm(df_ocr.img_id.unique(), desc="copy ocr imgs"):
    src, dst = TRAIN2_IMG_DIR/f, OCR_DIR/"images"/f
    if not dst.exists(): shutil.copy(src, dst)
print("✅ OCR dataset ready:", len(df_ocr))

# ---------- B · dataset ----------
CHARS = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"
c2i = {c:i+1 for i,c in enumerate(CHARS)} ; i2c={i+1:c for i,c in enumerate(CHARS)}
def enc(t): return [c2i[c] for c in t]

tfm = T.Compose([
    T.ToPILImage(),
    T.Resize((32,128)),
    T.ToTensor(),
    T.Normalize([0.5],[0.5])
])

class PlateDS(Dataset):
    def __init__(s, df, root): s.df, s.root = df, root
    def __len__(s): return len(s.df)
    def __getitem__(s,i):
        r=s.df.iloc[i]
        img=cv2.imread(str(s.root/"images"/r.img_id),0)
        return tfm(img), torch.tensor(enc(r.text)), len(r.text)

def col(batch):
    imgs,lbls,ll = zip(*batch)
    imgs = torch.stack(imgs)
    lbls = torch.cat(lbls)
    ll   = torch.tensor(ll)
    tl   = torch.full((len(batch),), imgs.size(3)//4, dtype=torch.long)
    return imgs, lbls, tl, ll

mask = np.random.rand(len(df_ocr))<0.1
train_dl = DataLoader(PlateDS(df_ocr[~mask], OCR_DIR),64,True,collate_fn=col)
val_dl   = DataLoader(PlateDS(df_ocr[mask],  OCR_DIR),64,False,collate_fn=col)

# ---------- C · CRNN ----------
class CRNN(nn.Module):
    def __init__(self, n):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1,64,3,1,1), nn.ReLU(), nn.MaxPool2d(2,2),
            nn.Conv2d(64,128,3,1,1), nn.ReLU(), nn.MaxPool2d(2,2),
            nn.Conv2d(128,256,3,1,1), nn.ReLU(), nn.MaxPool2d((2,1)),
            nn.Conv2d(256,256,3,1,1), nn.ReLU(), nn.MaxPool2d((2,1))
        )
        self.rnn = nn.LSTM(256,256,2,batch_first=True,bidirectional=True)
        self.fc  = nn.Linear(512,n+1)
    def forward(self,x):
        x = self.cnn(x)                        # [B,256,H=2,W]
        x = F.adaptive_avg_pool2d(x,(1,None))  # [B,256,1,W]
        x = x.squeeze(2)                       # [B,256,W]
        x = x.permute(0,2,1)                   # [B,W,256]
        x, _ = self.rnn(x)                     # [B,W,512]
        return self.fc(x).log_softmax(2)

device="cuda" if torch.cuda.is_available() else "cpu"
ocr   = CRNN(len(CHARS)).to(device)
ctc   = nn.CTCLoss(blank=0, zero_infinity=True)
opt   = torch.optim.AdamW(ocr.parameters(),1e-3)

# ---------- D · train ----------
EPOCHS=15
for e in range(EPOCHS):
    ocr.train(); tot=0
    for imgs,lbls,tl,ll in train_dl:
        imgs = imgs.to(device)
        opt.zero_grad()
        loss = ctc(ocr(imgs).permute(1,0,2), lbls, tl, ll)
        loss.backward(); opt.step()
        tot += loss.item()
    print(f"Epoch {e+1}/{EPOCHS}  loss={tot/len(train_dl):.4f}")

torch.save(ocr.state_dict(), ROOT/"crnn_lp.pth")
print("✔ CRNN trained →", ROOT/"crnn_lp.pth")


In [ ]:
#Cell 5: Prepare OCR Dataset and Train CRNN Model
# OCR training: prepare dataset and train CRNN
# %% 5 · FAST INFERENCE (DETECT + OCR) ON test_images
from pathlib import Path
import glob, os, cv2, pandas as pd, torch
from ultralytics import YOLO
import torchvision.transforms as T
from PIL import Image
from tqdm import tqdm

# ---------- 5‑A · choose detector weights ----------
run_ckpts = glob.glob(str(ROOT / "runs/detect/*/*.pt"))
best_pt   = [p for p in run_ckpts if p.endswith("best.pt")]
last_pt   = [p for p in run_ckpts if p.endswith("last.pt")]

if best_pt:
    det_wt = max(best_pt, key=os.path.getmtime)
    print("Using best.pt:", det_wt)
elif last_pt:
    det_wt = max(last_pt, key=os.path.getmtime)
    print("best.pt not found; using last.pt:", det_wt)
else:
    det_wt = "yolov8n.pt"
    print("No run checkpoints found; falling back to yolov8n.pt (pretrained)")

# ---------- 5‑B · EasyOCR (primary OCR) ----------
try:
    import easyocr
    reader = easyocr.Reader(["en"], gpu=torch.cuda.is_available())
    print("easyocr loaded – will use as primary OCR")
except ModuleNotFoundError:
    reader = None
    print("easyocr not installed – CRNN only")

# ---------- 5‑C · load CRNN weights ----------
ocr = CRNN(len(CHARS)).to(device)
crnn_path = ROOT / "crnn_lp.pth"
if crnn_path.exists():
    ocr.load_state_dict(torch.load(crnn_path, map_location=device))
    print("CRNN weights loaded:", crnn_path.name)
else:
    print("CRNN weights not found – will output blank if EasyOCR fails")
ocr.eval()

# ---------- 5‑D · detector & transforms ----------
det = YOLO(det_wt)
det.overrides["imgsz"] = 416            # smaller input → faster

tf = T.Compose([
    T.ToPILImage(),
    T.Resize((32, 128)),                # match CRNN training size
    T.ToTensor(),
    T.Normalize([0.5], [0.5])
])

def decode_crnn(img_bgr):
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    tensor = tf(gray).unsqueeze(0).to(device)
    with torch.no_grad():
        pred = ocr(tensor).permute(1, 0, 2).softmax(2).argmax(2).squeeze().cpu().numpy()
    txt, prev = "", -1
    for p in pred:
        if p != prev and p != 0:
            txt += i2c[p]
        prev = p
    return txt

# ---------- 5‑E · run batch inference ----------
rows = []
results = det.predict(
    source = str(TEST_IMG_DIR),        # folder of 201 images
    stream = True,                     # generator
    imgsz  = 416,
    verbose = False
)

total_imgs = len(list(TEST_IMG_DIR.glob("*.jpg")))
for r in tqdm(results, total=total_imgs, desc="inferring"):
    img_path = Path(r.path)
    if len(r.boxes) == 0:
        continue
    i = r.boxes.conf.argmax()
    xmin, ymin, xmax, ymax = map(int, r.boxes.xyxy[i].cpu())
    crop = r.orig_img[ymin:ymax, xmin:xmax]how to download the .ipynb and

    # ---- OCR pipeline ----
    plate_txt = "".join(reader.readtext(crop, detail=0)) if reader else ""
    if plate_txt == "":
        plate_txt = decode_crnn(crop)

    rows.append(dict(
        image_name = img_path.name,
        ymin = ymin, xmin = xmin,
        ymax = ymax, xmax = xmax,
        plate_text = plate_txt
    ))

out_df = pd.DataFrame(rows)
import csv
out_df.to_csv(PRED_CSV, index=False, quoting=csv.QUOTE_ALL, encoding="utf-8")
print(f"✅ Predictions saved → {PRED_CSV}")
display(out_df.head())
